### How many bikes were checked out in a specific timeframe?

#### predicting a demand (continuous number) - regression problem (for regression problems we need a different metric):

**Root Mean Squared Logarithmic Error**

note: this week it is better to overshoot with the model! (e.g. they will put 2 bikes too much at one station)

In [1]:
import pandas as pd

### Import data

In [2]:
df = pd.read_csv('train.csv')
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


### Convert datetime col to datetime format and add columns for day, month, dayofweek, year, dayofyear 

In [3]:
df['datetime'] = pd.to_datetime(df['datetime']) # convert column

df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour
df['dayofyear'] = df['datetime'].dt.dayofyear

df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,month,dayofweek,hour,dayofyear
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1,5,0,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1,5,1,1


In [4]:
df.shape

(10886, 16)

In [5]:
        
#df.drop(df[condition]. index, axis=0, inplace=True)

cond = df['weather'] == 4
df.drop(df[cond].index, axis=0, inplace=True)

### Train-test split



In [6]:
X = df.drop('count', axis=1)

type(X) # feature matrix



pandas.core.frame.DataFrame

In [7]:
X.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,month,dayofweek,hour,dayofyear
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,1,5,0,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,1,5,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,1,5,2,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,1,5,3,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,5,4,1


In [8]:
X.shape

(10885, 15)

In [9]:
y = pd.to_numeric(df['count'])
type(y) # series 

pandas.core.series.Series

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # stratify=y

# reset indexes
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)

y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

# check shapes
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(8708, 16) (8708,) (2177, 16) (2177,)


### Feature Engineering Function

notes:
#### if no clear measurable distance between the categories (e.g. seasons) = then one hot encoding
#### one hot encoding for categories where order is not important, ordinal encoding where you think order matters (as linear regression WILL take up ordinal encoding)

In [11]:
import datetime as dt
def feature_engineer(df): # take any dataframe, no matter if test or train
        # select relevant features
        df_sub = df[['hour', 'atemp', 'temp', 'humidity', 'month', 'workingday', 'weather']] # , 'windspeed', 'weather', 'month',
        
        df_sub2 = df_sub[['humidity', 'atemp', 'workingday', 'temp']]
        
        # one hot-encoding of season
        season_binary_df = pd.get_dummies(df_sub['month'], prefix='month') 
        season_binary_df = season_binary_df.drop('month_1', axis=1)
        
        # one hot encoding of weather cat
        weat_binary_df = pd.get_dummies(df_sub['weather'], prefix='weather_cat')
        weat_binary_df = weat_binary_df.drop('weather_cat_1', axis=1)
        
        # one hot encoding of hour
        hour_binary_df = pd.get_dummies(df_sub['hour'], prefix='hour')
        hour_binary_df = hour_binary_df.drop('hour_0', axis=1)
        
        # join with the sub_df
        df_fe = pd.DataFrame(df_sub2.join([weat_binary_df, season_binary_df, hour_binary_df], how='left'))
        
        # interaction term humidity and temperature
        df_fe['temp_hum_interact'] = df_fe['temp'] * df_fe['humidity']
        # drop temp col
        #df_fe = df_fe.drop('temp', axis=1)
        
        # interaction term working day and roushhours (6-9, 16-19)
        df_fe['workday_hour_7_interact'] = df_fe['workingday'] * hour_binary_df['hour_7'] 
        df_fe['workday_hour_8_interact'] = df_fe['workingday'] * hour_binary_df['hour_8'] 
        df_fe['workday_hour_9_interact'] = df_fe['workingday'] * hour_binary_df['hour_9']
        df_fe['workday_hour_17_interact'] = df_fe['workingday'] * hour_binary_df['hour_17']  
        df_fe['workday_hour_18_interact'] = df_fe['workingday'] * hour_binary_df['hour_18']
        df_fe['workday_hour_19_interact'] = df_fe['workingday'] * hour_binary_df['hour_19']
        
        # interaction term non-working day and hour of day 
        # create non-working day column
        df_sub['non_workingday'] = df_sub['workingday'].replace({0:1, 1:0})
        df_fe['non_workingday'] = df_sub['non_workingday'] * hour_binary_df['hour_11'] 
        df_fe['non_workingday'] = df_sub['non_workingday'] * hour_binary_df['hour_12'] 
        df_fe['non_workingday'] = df_sub['non_workingday'] * hour_binary_df['hour_13']
        df_fe['non_workingday'] = df_sub['non_workingday'] * hour_binary_df['hour_14']

        
        
        # reset index
        df_fe.reset_index()

        
        return df_fe

In [12]:
X_train_fe = feature_engineer(X_train)

In [13]:
X_train_fe.head()

,humidity,atemp,workingday,temp,weather_cat_2,weather_cat_3,month_2,month_3,month_4,month_5,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,temp_hum_interact
0,75,15.910,0,11.48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,861.00
1,61,32.575,0,28.70,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1750.70
2,59,38.635,1,32.80,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1935.20
3,65,13.635,1,12.30,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,799.50
4,62,34.090,0,29.52,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1830.24


In [14]:
X_train_fe.shape

(8708, 41)

### Polynomial Features 
**has to be done BEFORE SCALING**
**mostly done on continous data (like temperature, humidity)

In [15]:
from sklearn.preprocessing import PolynomialFeatures
pt = PolynomialFeatures()

p_features = pt.fit_transform(X_train_fe[['atemp']])
#p_features[:,2]
p_features.shape

(8708, 3)

In [16]:
polynomial_temp_df = pd.DataFrame.from_records(p_features)


polynomial_temp_df.columns = ['t', 't2', 'temp_pol2']
polynomial_temp_df.head()

,t,t2,temp_pol2
0,1.0,15.910,253.128100
1,1.0,32.575,1061.130625
2,1.0,38.635,1492.663225
3,1.0,13.635,185.913225
4,1.0,34.090,1162.128100


In [17]:
polynomial_temp_df.shape

(8708, 3)

In [18]:
X_train_fe = X_train_fe.join(polynomial_temp_df['temp_pol2'], how='left')
X_train_fe.head()

,humidity,atemp,workingday,temp,weather_cat_2,weather_cat_3,month_2,month_3,month_4,month_5,...,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,temp_hum_interact,temp_pol2
0,75,15.910,0,11.48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,861.00,253.128100
1,61,32.575,0,28.70,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1750.70,1061.130625
2,59,38.635,1,32.80,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1935.20,1492.663225
3,65,13.635,1,12.30,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,799.50,185.913225
4,62,34.090,0,29.52,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1830.24,1162.128100


In [19]:
X_train_fe.shape

(8708, 42)

In [20]:
X_train_fe = X_train_fe.reset_index()

In [21]:
##X_train_fe = X_train_fe.fillna(0)

In [22]:
X_train_fe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8708 entries, 0 to 8707
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              8708 non-null   int64  
 1   humidity           8708 non-null   int64  
 2   atemp              8708 non-null   float64
 3   workingday         8708 non-null   int64  
 4   temp               8708 non-null   float64
 5   weather_cat_2      8708 non-null   uint8  
 6   weather_cat_3      8708 non-null   uint8  
 7   month_2            8708 non-null   uint8  
 8   month_3            8708 non-null   uint8  
 9   month_4            8708 non-null   uint8  
 10  month_5            8708 non-null   uint8  
 11  month_6            8708 non-null   uint8  
 12  month_7            8708 non-null   uint8  
 13  month_8            8708 non-null   uint8  
 14  month_9            8708 non-null   uint8  
 15  month_10           8708 non-null   uint8  
 16  month_11           8708 

### Scaling

In [23]:
# # scaling with min max
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train_fe) # memorizes the min and max for each column, no y 
# X_train_fe = scaler.transform(X_train_fe) # does the actual scaling; still no y
# X_train_fe  # numpy array

In [24]:
# # scaling with standard scaler
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_fe = scaler.fit_transform(X_train_fe)
# X_train_fe

In [25]:
X_train_fe.shape

(8708, 43)

### Train model: Linear regression with scikit-learn

In [26]:
from sklearn.linear_model import LinearRegression

### Fit the model

In [27]:
# Create the model 
m = LinearRegression(normalize=True)


In [28]:
m.fit(X_train_fe, y_train)


### Reduce the complexity through regularization
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet # combination of Ridge and Lasso

m_ridge = Ridge(alpha=5.0)
m_ridge.fit(X_train_fe, y_train)

m_lasso = Lasso(alpha=5.0)
m_lasso.fit(X_train_fe, y_train)

m_ridge_lasso = ElasticNet(alpha=0.5)
m_ridge_lasso.fit(X_train_fe, y_train)

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [29]:
m.coef_

array([-5.80508029e-05,  1.27508780e+00,  2.32442111e+00,  3.01078158e+00,
        1.04601809e+01, -5.49752129e+00, -6.50369266e+01,  5.88224193e+00,
        2.09857442e+01,  3.42289158e+01,  6.72074516e+01,  4.46502468e+01,
        1.81884863e+01,  3.31530375e+01,  7.86398117e+01,  9.01656797e+01,
        7.35328511e+01,  6.93363196e+01, -1.63018546e+01, -2.61303159e+01,
       -3.76291450e+01, -3.83833910e+01, -2.18274181e+01,  3.65573404e+01,
        1.70397706e+02,  3.07119625e+02,  1.61652722e+02,  1.02119027e+02,
        1.27114690e+02,  1.60327154e+02,  1.58102932e+02,  1.32796210e+02,
        1.48188115e+02,  2.08848160e+02,  3.65427779e+02,  3.36158820e+02,
        2.34425252e+02,  1.52885402e+02,  1.07849313e+02,  7.00873732e+01,
        3.38454124e+01, -1.20810938e-01, -1.42022784e-03])

In [30]:
# Coefficients
w_0 = m.intercept_
w_1 = m.coef_[0]

In [31]:
# Interpretation of w_0
w_0, w_1

(-163.90585140648275, -5.805080291258868e-05)

Evaluate/Optimize the model

    What kind of evaluation metrics can we use?
        MSE
        RMSLE (root mean squared logarithmic error)
        R-squared (coefficient of determination)
    You should do cross-validation (on your own)



In [32]:
# Look at the training score
# Return the coefficient of determination R^2 of the prediction.b
m.score(X_train_fe, y_train) # 
# R-squared 0.6357555992294879
# incl weather cat one hot encoded 0.6389685222446879
# incl interaction working day and hour_18  0.6498956851149124
# incl interaction working day and hour_17  0.6615055908463631
# incl interaction working day and hour_19  0.6649612525052413
# incl interaction working day and hour_7   0.6850735381594927
# incl interaction working day and hour_8   0.7295004828181124
# incl interaction working day and hour_9   0.7333398004517597
# incl interaction non working day and hour_11   0.7366122513219515
# incl interaction non working day and hour_12   0.7376443860278101
# incl interaction non working day and hour_13   0.7378722116507113
# incl interaction non working day and hour_14   0.7387041310756237
# incl temperature polynomial, degrees of freedom = 2    0.7457480384640465

0.6426815156306307

In [33]:
# with regularization
print('m_ridge\n',
m_ridge.score(X_train_fe, y_train),
      '\nm_lasso\n',
m_lasso.score(X_train_fe, y_train),
      '\nm_ridge_lasso\n',
m_ridge_lasso.score(X_train_fe, y_train))



m_ridge
 0.641300373845621 
m_lasso
 0.3848511141284644 
m_ridge_lasso
 0.3526505647051408


### Optimize/ Cross Validation

In [34]:
from sklearn.model_selection import cross_val_score

In [35]:
cross_val_result_m_negMSR = cross_val_score(m, X_train_fe, y_train, cv=10, scoring='neg_mean_squared_error')
cross_val_result_m_negMSR

array([-12453.66882878, -12248.2318871 , -11366.12662909, -11502.44638904,
       -11852.61488855, -11628.09134694, -12259.7786323 , -12126.63911227,
       -11619.27765099, -10351.82024585])

In [36]:
cross_val_result_m_r2 = cross_val_score(m, X_train_fe, y_train, cv=10, scoring='r2')
cross_val_result_m_r2

array([0.59346452, 0.62597329, 0.6598693 , 0.65778061, 0.64403531,
       0.60999316, 0.63251097, 0.64801508, 0.64567823, 0.6639016 ])

In [37]:
print('cross_val_result_m_negMSR mean',
cross_val_result_m_negMSR.mean(), 
      '\ncross_val_result_m_r2 mean',
      cross_val_result_m_r2.mean())   # no overfitting !

cross_val_result_m_negMSR mean -11740.869561091802 
cross_val_result_m_r2 mean 0.6381222060796742


### Predictions

In [38]:
# Make predictions for the training data
y_pred_train = m.predict(X_train_fe)
y_pred_train

array([ 57.94528441, 315.75586545, 480.6315465 , ..., 350.0533085 ,
       151.8075638 , 538.97057893])

### Calculate Test Score

In [39]:
X_test_fe = feature_engineer(X_test)



In [40]:
X_test_fe

,humidity,atemp,workingday,temp,weather_cat_2,weather_cat_3,month_2,month_3,month_4,month_5,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,temp_hum_interact
0,60,26.515,1,22.96,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1377.60
1,89,29.545,1,27.06,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2408.34
2,54,21.210,1,17.22,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,929.88
3,44,31.820,1,27.88,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1226.72
4,52,25.000,1,21.32,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1108.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,78,30.305,1,27.06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2110.68
2173,88,21.210,1,17.22,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1515.36
2174,51,11.365,1,8.20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,418.20
2175,36,37.120,1,34.44,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1239.84


In [41]:
pt = PolynomialFeatures(degree=2)
p_features = pt.fit_transform(X_test_fe[['atemp']])
polynomial_temp_df = pd.DataFrame.from_records(p_features)


polynomial_temp_df.columns = ['t', 't2', 'temp_pol2']
X_test_fe = X_test_fe.join(polynomial_temp_df['temp_pol2'], how='left')
#X_test_fe = X_test_fe.fillna(0)
X_test_fe = X_test_fe.reset_index()

In [42]:
# # scaling with standard scaler
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_test_fe_sc = scaler.fit_transform(X_test_fe)

In [43]:
X_test_fe.shape

(2177, 43)

In [44]:
y_test.shape

(2177,)

In [45]:
m.score(X_test_fe, y_test)

0.6403951212014641

### Kaggle submission

Kaggle evaluates the results of all submissions based on the Root Mean Squared Log Error (RMSLE).

The purpose of this metric is to treat the error in relation to the bike count. If the amount of bikes is 100, an error of 10 bikes does not matter that much, but if the predicted value is only 10, the same error is a lot. The logarithm fixes that.

To optimize your model against the RMSLE, you should take the logarithm of the target colum (y). Because 0 is a valid target value, use the log of y+1
instead:

In [46]:
import numpy as np
ylog = np.log1p(y_test)
ylog

0       5.141664
1       6.525030
2       6.111467
3       5.955837
4       5.579730
          ...   
2172    2.079442
2173    5.831882
2174    1.609438
2175    6.210600
2176    6.018593
Name: count, Length: 2177, dtype: float64

In [47]:
from sklearn.metrics import mean_squared_log_error

np.sqrt(mean_squared_log_error(y_pred))

NameError: name 'y_pred' is not defined